<a href="https://colab.research.google.com/github/q759729997/test_pytorch/blob/master/linear_regression_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 现行回归的简洁实现

In [0]:
import torch
import numpy as np

In [0]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0,1,(num_examples, num_inputs)),dtype=torch.float)
labels = true_w[0]*features[:,0]+true_w[1]*features[:,1]+true_b
labels += torch.tensor(np.random.normal(0,0.01,size=labels.size()),dtype=torch.float)

In [11]:
features.size()

torch.Size([1000, 2])

In [12]:
labels.size()

torch.Size([1000])

In [0]:
# 读取数据
batch_size = 10
# 将训练数据的特征和标签组合
dataset = torch.utils.data.TensorDataset(features,labels)
# 随机读取小批量
data_iter = torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)

In [14]:
for X,y in data_iter:
  print(X,y)
  break

tensor([[-2.2198, -0.6570],
        [ 0.1922, -1.4269],
        [ 0.5215, -0.4237],
        [-1.5246,  0.6838],
        [ 0.1469, -1.0290],
        [ 0.3899,  0.5862],
        [ 2.1282, -1.2686],
        [-0.7979, -1.0947],
        [-2.9282, -0.1944],
        [-1.0370, -1.8211]]) tensor([ 2.0147,  9.4344,  6.6825, -1.1695,  7.9917,  2.9668, 12.7587,  6.3389,
        -0.9830,  8.3058])


In [15]:
# 定义模型
class LinearNet(torch.nn.Module):
  def __init__(self,n_feature):
    super(LinearNet,self).__init__()
    self.linear = torch.nn.Linear(n_feature,1)
  # forward定义前向传播
  def forward(self,x):
    y = self.linear(x)
    return y

net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [17]:
# 查看参数
for param in net.parameters():
  print(param)

Parameter containing:
tensor([[0.4655, 0.4406]], requires_grad=True)
Parameter containing:
tensor([-0.0959], requires_grad=True)


In [24]:
# 定义损失函数
loss = torch.nn.MSELoss()
print(loss)

MSELoss()


In [25]:
# 定义优化算法
optimizer = torch.optim.SGD(net.parameters(),lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [27]:
# 训练模型
num_epochs = 10
for epoch in range(1,num_epochs+1):
  for X,y in data_iter:
    output = net(X)
    l = loss(output,y.view(-1,1))
    optimizer.zero_grad() # 梯度清零
    l.backward()
    optimizer.step()
  print('epoch:{}, loss:{}'.format(epoch,l))

epoch:1, loss:0.00021883817680645734
epoch:2, loss:0.00017317570745944977
epoch:3, loss:0.00012074378173565492
epoch:4, loss:0.0001122339308494702
epoch:5, loss:0.0001308737409999594
epoch:6, loss:9.369711187900975e-05
epoch:7, loss:0.00011131448263768107
epoch:8, loss:4.063736560055986e-05
epoch:9, loss:4.807809818885289e-05
epoch:10, loss:0.00013224477879703045


In [28]:
# 查看模型学到的参数
dense = net.linear
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9996, -3.4004]], requires_grad=True)
4.2 Parameter containing:
tensor([4.2000], requires_grad=True)
